<a href="https://colab.research.google.com/github/stefanusaw/Introduction_to_GenAI/blob/main/Project_Neural_Networks_and_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets

In [2]:
pip install tensorflow

In [3]:
from datasets import load_dataset

ds = load_dataset("microsoft/cats_vs_dogs")
print(ds)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 23410
    })
})


In [15]:
from PIL import Image
import numpy as np
import tensorflow as tf

# Function to preprocess the image
def preprocess_example(example):
    # Convert image to a format TensorFlow can use (e.g., resized and normalized)
    image = example['image'].resize((150, 150))  # Resize to 150x150 pixels
    image = np.array(image)  # Convert to a numpy array
    image = image / 255.0  # Normalize the image to [0, 1]
    label = example['labels']  # Binary label (0 for cats, 1 for dogs)

    # Apply random augmentations
    image = tf.image.random_flip_left_right(image)  # Random horizontal flip
    image = tf.image.random_flip_up_down(image)  # Random vertical flip
    image = tf.image.random_brightness(image, max_delta=0.2)  # Random brightness
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)  # Random contrast

    # Return as a dictionary
    return {'image': image, 'labels': label}


train_subset = ds['train'].select(range(1500))
val_subset = ds['train'].select(range(1500, 1800))

# Preprocess the data
train_subset = train_subset.map(preprocess_example)
val_subset = val_subset.map(preprocess_example)

# Convert to TensorFlow datasets (using from_generator)
def convert_to_tf(dataset):
    return tf.data.Dataset.from_generator(
        lambda: ((x['image'], x['labels']) for x in dataset),  # Yield image, label pairs
        output_signature=(
            tf.TensorSpec(shape=(150, 150, 3), dtype=tf.float32),  # Image shape and dtype
            tf.TensorSpec(shape=(), dtype=tf.int64)                # Labels shape and dtype
        )
    )

# Convert the datasets to TensorFlow datasets
train_ds_tf = convert_to_tf(train_subset)
val_ds_tf = convert_to_tf(val_subset)

# Batch and prefetch the datasets
train_ds_tf = train_ds_tf.batch(32).prefetch(tf.data.experimental.AUTOTUNE)
val_ds_tf = val_ds_tf.batch(32).prefetch(tf.data.experimental.AUTOTUNE)


Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [23]:
import tensorflow as tf

# Define the augmentation function for individual images
def augment_image(image):
    image = tf.image.random_flip_left_right(image)  # Horizontal flip
    image = tf.image.random_flip_up_down(image)     # Vertical flip
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)  # Random contrast
    image = tf.image.random_brightness(image, max_delta=0.2)  # Random brightness
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)  # Random saturation
    image = tf.image.random_hue(image, max_delta=0.1)  # Random hue

    # Ensure the image is resized to the correct shape
    image = tf.image.resize(image, [150, 150])

    return image

In [27]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Build the model
model = models.Sequential([
    # Conv2D layer with L2 regularization
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3),
                  kernel_regularizer=regularizers.l2(0.01)),  # Apply L2 regularization
    layers.MaxPooling2D((2, 2)),

    # Another Conv2D layer with L2 regularization
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D((2, 2)),

    # Dropout layer
    layers.Dropout(0.5),  # Drop 50% of the neurons to prevent overfitting

    # Flatten the output from the convolutional layers
    layers.Flatten(),

    # Dense layer with L2 regularization
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),

    # Dropout layer
    layers.Dropout(0.5),  # Drop 50% of the neurons again

    # Output layer
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Initialize the ImageDataGenerator with augmentations
datagen = ImageDataGenerator(
    rotation_range=20,  # Random rotations between -20 and +20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally (20% of image width)
    height_shift_range=0.2,  # Randomly shift images vertically (20% of image height)
    shear_range=0.2,  # Apply random shear transformations
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill in any missing pixels after transformations
)


# Now apply the augmentations to each image in the dataset
def apply_augmentation(x, y):
    x = tf.map_fn(augment_image, x, dtype=tf.float32)  # Apply augmentations to each image in the batch
    return x, y

# Apply the augmentations to the training dataset
train_ds_augmented = train_ds_tf.map(lambda x, y: apply_augmentation(x, y))


# Train the model
history = model.fit(
    train_ds_augmented,
    validation_data=val_ds_tf,  # Preprocessed validation data
    epochs=20,  # Number of epochs
    batch_size=32,
    callbacks=[early_stopping]  # Early stopping callback to prevent overfitting
)


Epoch 1/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 209s 4s/step - accuracy: 0.9826 - loss: 1.7241 - val_accuracy: 1.0000 - val_loss: 0.2586
Epoch 2/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 205s 4s/step - accuracy: 1.0000 - loss: 0.1672 - val_accuracy: 1.0000 - val_loss: 0.0398
Epoch 3/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 208s 4s/step - accuracy: 1.0000 - loss: 0.0306 - val_accuracy: 1.0000 - val_loss: 0.0138
Epoch 4/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 207s 4s/step - accuracy: 1.0000 - loss: 0.0116 - val_accuracy: 1.0000 - val_loss: 0.0066
Epoch 5/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 207s 4s/step - accuracy: 1.0000 - loss: 0.0059 - val_accuracy: 1.0000 - val_loss: 0.0042
Epoch 6/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 205s 4s/step - accuracy: 1.0000 - loss: 0.0036 - val_accuracy: 1.0000 - val_loss: 0.0025
Epoch 7/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 205s 4s/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy: 1.0000 - val_loss: 0.0058
Epoch 8/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 205s 4s/step - accuracy: 1.0000 - loss: 0.0046 - val_accuracy: 1.0000 - v

In [28]:

test_subset = ds['train'].select(range(1800, 2000))

# Preprocess the data
test_subset = test_subset.map(preprocess_example)

# Convert the datasets to TensorFlow datasets
test_ds_tf = convert_to_tf(test_subset)

# Batch and prefetch the datasets
test_ds_tf = test_ds_tf.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [29]:
test_loss, test_acc = model.evaluate(test_ds_tf)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - accuracy: 1.0000 - loss: 9.7518e-04
Test accuracy: 1.0
Test loss: 0.0009751844336278737


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


In [30]:
model.save('cats_vs_dogs_model.h5')

In [31]:
model.save('cats_vs_dogs_model.keras')